In [1]:
import rmcalyse
from rmcalyse import rmc_data

import numpy as np
import matplotlib.pyplot as plt
import time



 8888888b. 888b     d888 .d8888b.         888                        
 888   Y88b8888b   d8888d88P  Y88b        888                        
 888    88888888b.d88888888    888        888                        
 888   d88P888Y88888P888888        8888b. 888888  888.d8888b  .d88b. 
 8888888P" 888 Y888P 888888           "88b888888  88888K     d8P  Y8b
 888 T88b  888  Y8P  888888    888.d888888888888  888"Y8888b.88888888
 888  T88b 888   "   888Y88b  d88P888  888888Y88b 888     X88Y8b.    
 888   T88b888       888 "Y8888P" "Y888888888 "Y88888 88888P' "Y8888 
                                                   888               
                                              Y8b d88P               
                                               "Y88P"                
 ------------- A toolbox for analysing RMCProfile outputs -----------
 --------------------------------------------------------------------
 ------------------------ Anton Goetzee-Barral ----------------------
 -----------------

In [2]:
position_labels = rmc_data.position_labels
orthonormal_positions = rmc_data.orthonormal_positions

In [3]:
center_atom = ['O']
orbit_atom = ['Ti']
max_d = 4 # max distance for centroid polyhedra
coordination_no = 8

In [4]:
# Uses orthonormalised distances
def array_distance(positions_labels, orthonormal_positions, center_atom):
    positions = np.array([x for x in orthonormal_positions]) # want a numpy array of the positions
    offset = rmc_data.cell_parameters[0]/2 # half a unit cell (helpfully it's cubic so don't need anything more complex than this)
    positions -= offset # now the unit cell goes from -19 to 19
    idx_of_interest = [i for i,x in enumerate(orthonormal_positions) if position_labels[i][1] in center_atom] # hack to find a list of indices we're interested in
    results = np.zeros((positions.shape[0],3)) # predefine results array for speed
    all_distances = []

    t0 = time.perf_counter()
    for i in idx_of_interest:
        shifted = (positions - positions[i]) #shift all atoms so this one is at the origin
        reshuffled = np.mod(shifted+offset, offset*2)-offset # the clever moving of atoms outside -19:19 back
        distances  = np.power(np.square(reshuffled).sum(1),0.5) # calculate distances
        idx = np.argsort(distances) # returns an array of sorted indices (i.e. distances[idx[0]] = 0, the distance from the atom to itself, distances[idx[1]] is somewthing like 2.5, etc.
        results[i,:] = -np.mean(reshuffled[idx[1:7], :],0) #note we start at 1 to avoid the 0 for the same atom. Then average them. this calculates the central point so the displacements vector is -ve this
        all_distances.append(distances)
    runtime = round(time.perf_counter() - t0, 5)
    print(f'total time taken {runtime} s')
    return distances, runtime
    

distances_no = array_distance(position_labels, orthonormal_positions, ['Sr', 'Ti', 'O', 'Nb'])

total time taken 0.00404 s


In [5]:
def dummy_data(no_atoms):
    np.random.seed(42)
    dummy_positions = np.random.rand(no_atoms,3)
    matrix = np.random.rand(3,3)
    return dummy_positions, matrix

In [6]:
def plot_times(max_atoms, step):
    times = []
    no_atoms = []
    
    for atom_no in range(0, max_atoms + step, step):
        t0 = time.perf_counter()
        for i in range(max_atoms):
            positions = dummy_cell(atom_no)
            orht_pos = np.dot(matrix, positions.T).T

        runtime = time.perf_counter() - t0
        times.append(runtime)
        no_atoms.append(atom_no)

    total_time = round(sum(times), 3)
    plt.scatter(np.array(no_atoms), times)
    plt.xlabel('Number of atoms')
    plt.ylabel('Time to compute orthonormalisation (s)')
    plt.title(f'''total time {total_time} s
max atoms {atom_no}''')

    plt.show()   



In [8]:
t0 = time.perf_counter()
orht_pos = np.dot(matrix, dummy_cell(100000000).T).T
print(f'{round(time.perf_counter() - t0, 4)} s')

NameError: name 'matrix' is not defined

In [9]:
def array_distance_orthonormaliser(labels, positions, center_atom):
       
    offset = 0.5 # half a unit cell
    
    positions -= offset # now the unit cell goes from -0.5:0.5
    
    # list of indices we're interested in
    idx_of_interest = [i for i,x in enumerate(positions) if labels[i][1] in center_atom]
    
    all_distances = []
    all_labels = []
    
    for index in idx_of_interest:
        
        shifted = (positions - positions[index]) #shift all atoms so this one is at the origin
        
        reshuffled = np.mod(shifted + offset, offset * 2) - offset # the clever moving of atoms outside -0.5:0.5
        
        # ORTHONORMALISATION
        orthonormalised = np.dot(rmc_data.matrix, reshuffled.T).T
        
        distances  = np.power(np.square(orthonormalised).sum(1),0.5) # calculate distances
        
        all_distances.append(distances)
    
    return all_distances


In [10]:
all_distances = array_distance_orthonormaliser(position_labels, rmc_data.raw_basis_positions, 'Sr')

In [11]:
np.array(all_distances).flatten()

array([0.        , 4.        , 4.        , 5.65685425, 4.        ,
       5.65685425, 5.65685425, 6.92820323, 3.46410162, 3.46410162,
       3.46410162, 3.46410162, 3.46410162, 3.46410162, 3.46410162,
       3.46410162, 2.82842712, 2.82842712, 2.82842712, 4.89897949,
       2.82842712, 2.82842712, 2.82842712, 2.82842712, 4.89897949,
       4.89897949, 2.82842712, 4.89897949, 2.82842712, 4.89897949,
       2.82842712, 4.89897949, 4.89897949, 2.82842712, 2.82842712,
       4.89897949, 4.89897949, 4.89897949, 4.89897949, 4.89897949,
       4.        , 0.        , 5.65685425, 4.        , 5.65685425,
       4.        , 6.92820323, 5.65685425, 3.46410162, 3.46410162,
       3.46410162, 3.46410162, 3.46410162, 3.46410162, 3.46410162,
       3.46410162, 4.89897949, 2.82842712, 2.82842712, 2.82842712,
       2.82842712, 2.82842712, 4.89897949, 2.82842712, 4.89897949,
       2.82842712, 2.82842712, 4.89897949, 4.89897949, 4.89897949,
       2.82842712, 2.82842712, 4.89897949, 2.82842712, 4.89897

In [20]:
def array_distance_orthonormaliser(labels, positions, matrix, center_atom, orbit_atom):
       
    offset = 0.5 # half a unit cell
    
    positions -= offset # now the unit cell goes from -0.5:0.5
    
    # list of indices we're interested in
    idx_of_interest = [i for i,x in enumerate(positions) if labels[i][1] in center_atom]
    idxB = [i for i,x in enumerate(positions) if labels[i][1] in orbit_atom]

    all_distances = []
    all_labels = []
    
    for index in idx_of_interest:
        
        shifted = (positions[idxB] - positions[index]) # shift all atoms so this one is at the origin
        
        reshuffled = np.mod(shifted + offset, offset * 2) - offset # the clever moving of atoms outside -0.5:0.5
        
        # ORTHONORMALISATION
        orthonormalised = np.dot(matrix, reshuffled.T).T
        
        distances  = np.power(np.square(orthonormalised).sum(1),0.5) # calculate distances
        
        all_distances.append(distances)
        all_labels.append(labels[index])
        all_labels.append(labels[idxB])
    
    return all_distances, all_labels

In [27]:
all_distances, all_labels = distance_shuffle.array_distance_orthonormaliser(
    position_labels, 
    atom_positions,
    rmc_data.matrix, 
    'O', 
    'O')

In [28]:
all_distances

[array([0.        , 2.82842712, 2.82842712, 4.        , 2.82842712,
        2.82842712, 4.        , 4.89897949, 2.82842712, 5.65685425,
        4.89897949, 2.82842712, 4.        , 2.82842712, 4.89897949,
        5.65685425, 2.82842712, 4.89897949, 5.65685425, 4.89897949,
        4.89897949, 6.92820323, 4.89897949, 4.89897949]),
 array([2.82842712, 0.        , 2.82842712, 2.82842712, 4.        ,
        4.89897949, 4.89897949, 4.        , 2.82842712, 4.89897949,
        5.65685425, 4.89897949, 2.82842712, 4.        , 2.82842712,
        2.82842712, 5.65685425, 4.89897949, 4.89897949, 5.65685425,
        2.82842712, 4.89897949, 6.92820323, 4.89897949]),
 array([2.82842712, 2.82842712, 0.        , 2.82842712, 4.89897949,
        4.        , 2.82842712, 2.82842712, 4.        , 2.82842712,
        4.89897949, 5.65685425, 4.89897949, 2.82842712, 4.        ,
        4.89897949, 4.89897949, 5.65685425, 4.89897949, 2.82842712,
        5.65685425, 4.89897949, 4.89897949, 6.92820323]),
 array([4.